In [34]:
# packages

import numpy as np
import pandas as pd

In [2]:

FEE_HAIRCUT = 1 - 0.003
SWAP_CURVE_Y = np.arange(0.01, 1.0, 0.1 )
CHAIN_LIQUIDITY_TOK_2 = [2.0, 2.0, 2.0, 2.0]
CHAIN_LIQUIDITY_TOK_1 = [1.0, 1.0, 1.0, 1.0]
MAX_RATIO_OUTSWAP_TO_LIQUIDITY = 0.9
MAX_RATIO_INSWAP_TO_LIQUIDITY = 0.5

sample_swap_init_Y_str = "\tSwap curve Y: "
sample_chain_tok_1_str = "\tToken 1 liquidity along the chain:"
sample_chain_tok_2_str = "\tToken 2 liquidity along the chain:"
sample_fee_haircut_str = "\tFee haircut:"
print(f"Sample data \n"
      + sample_swap_init_Y_str + f"{SWAP_CURVE_Y} \n"
      + sample_chain_tok_1_str + f"{CHAIN_LIQUIDITY_TOK_1}\n"
      + sample_chain_tok_2_str + f"{CHAIN_LIQUIDITY_TOK_2}\n"
      + sample_fee_haircut_str + f"{FEE_HAIRCUT:6.4f}\n\n" )

Sample data 
	Swap curve Y: [0.01 0.11 0.21 0.31 0.41 0.51 0.61 0.71 0.81 0.91] 
	Token 1 liquidity along the chain:[1.0, 1.0, 1.0, 1.0]
	Token 2 liquidity along the chain:[2.0, 2.0, 2.0, 2.0]
	Fee haircut:0.9970




In [53]:
EXCHANGES = [
    {"input_token": 1, "input_liquidity": 2.0, "output_token": 2, "output_liquidity": 1.0, "haircut": FEE_HAIRCUT},
    {"input_token": 2, "input_liquidity": 2.0, "output_token": 3, "output_liquidity": 1.0, "haircut": FEE_HAIRCUT},
    {"input_token": 3, "input_liquidity": 2.0, "output_token": 4, "output_liquidity": 1.0, "haircut": FEE_HAIRCUT},
    {"input_token": 4, "input_liquidity": 2.0, "output_token": 1, "output_liquidity": 1.0, "haircut": FEE_HAIRCUT}, ]


In [27]:
SWAP_CURVE_X = np.exp(np.linspace(0, 1, 50)) - 1
#print(SWAP_CURVE_X)

In [48]:
# Define forward and backward swap calculators
from typing import Union

def forward_swap(x:Union[float, np.ndarray], haircut:float = FEE_HAIRCUT) -> Union[float, np.ndarray]:
    if isinstance(x, float):
        assert x >= 0
    elif isinstance(x, np.ndarray):
        assert np.all(x) >= 0
    assert haircut > 0
    return haircut * x /(1 + haircut * x)

def backward_swap(y:Union[float, np.ndarray], haircut:float = FEE_HAIRCUT) -> Union[float, np.ndarray]:
    if isinstance(y, float):
        assert y >= 0.0
        assert y < 1.0
    elif isinstance(y, np.ndarray):
        assert np.all(y) >= 0
        assert np.all(y < 1.0)
    assert haircut > 0
    return y /(haircut - haircut * y)



In [70]:
def generate_ITL(exchange_list:list[dict]=EXCHANGES) -> dict:
    ITL = {}
    for i in range(len(exchange_list)-1):
        assert exchange_list[i]['output_token'] == exchange_list[i+1]['input_token']
        ITL[exchange_list[i+1]['input_token']] = exchange_list[i]['output_liquidity']/ exchange_list[i+1]['input_liquidity']
    return ITL



In [75]:
ITL = generate_ITL()
print(ITL[2])
exchange_list = EXCHANGES

0.5


In [89]:
# Define swap curves
x_curve = SWAP_CURVE_X
y_curve = forward_swap(x_curve)
test_x_curve = backward_swap(y_curve)

# Check if forward_swap equals inverse of backward_swap
inverse_test = np.allclose(x_curve, test_x_curve)
print(f"Swap curves defined. \nForward swap is inverse of Backward swap: {inverse_test}")

Swap curves defined. 
Forward swap is inverse of Backward swap: True


In [105]:
# Recursive compute

# Exchange 1 -> 2
xi_1 = x_curve * exchange_list[0]['input_liquidity']   # a_{1,2} * \vv x
xi_indx = np.searchsorted(x_curve, xi_1, side='right') #
eta_1 = y_curve[xi_indx < 50]


# Token 2
x_curve1 = x_curve[xi_indx < 50]
print(len(eta_1))
xi_2 = eta_1 * ITL[2]
xi_indx = np.searchsorted(x_curve1, xi_2, side='right')
eta_2 = y_curve[xi_indx < 50]
#for i in range(len(eta_1)):
#    print(f"x: {x_curve[i]:6.4f}, xi_2: {xi_2[i]:6.4f}, xi_indx: {xi_indx[i]}, eta: {eta_2[i]:6.4f}")


31


IndexError: boolean index did not match indexed array along axis 0; size of axis is 50 but size of corresponding boolean axis is 31

In [38]:



def generate_upper_bounds(exch_data: list[dict], x_curve: np.ndarray, y_curve: np.ndarray) -> float:

    # delta_1 upper bound is MAX_RATIO_INSWAP_TO_LIQUIDITY
    exch_data[0]['delta_in'] = exch_data[0]['liq_in'] * MAX_RATIO_INSWAP_TO_LIQUIDITY

    # delta_2 upper bound is forward swap of delta_1
    exch_data[0]['upper_bound'] = exch_data[0]['liq_out'] * forward_swap(MAX_RATIO_INSWAP_TO_LIQUIDITY)

    # Eta initialized from delta_2 upper bound
    exch_data[0]['eta'] = exch_data[0]['upper_bound']

    print(f"Exchange data: {exch_data[0]}")

    # Eta and bound loop
    for i in range(1,len(exch_data)):
        exch_prior = exch_data[i-1]["exch_number"]
        exch_current = exch_data[i]['exch_number']
        exch_prior_out_liq = exch_data[i-1]['liq_out']
        exch_current_in_liq = exch_data[i]['liq_in']
        exch_current_out_liq = exch_data[i]['liq_out']

    #    print(f"Exchange number {exch_current}" )
    #    print(f"\t Prior exchange {exch_prior}" )
    #    print(f"\t Prior exchange outgoing leg liquidity: {exch_prior_out_liq}")
    #    print(f"\t Current exchange {exch_current}" )
    #    print(f"\t Current exchange incoming leg liquidity: {exch_current_in_liq}")

        # a(i, i-1)/a(i,i+1)
        ratio_out_div_by_in = exch_prior_out_liq / exch_current_in_liq
        xi_indx = np.searchsorted(x_curve, ratio_out_div_by_in, side='right')

     #   print(f"\t Ratio {ratio_out_div_by_in}")
     #   print(f"\t X value above {x_curve[xi_indx]}")
     #   print(f"\t X value below {x_curve[xi_indx-1]}")

        eta = float(y_curve[xi_indx]) * exch_data[i-1]['eta']
        exch_data[i]['upper_bound'] = eta *  exch_current_out_liq
        exch_data[i]['eta'] = eta

        print(f"Exchange data: {exch_data[i]}")

    return exch_data[len(exch_data)-1]['upper_bound']



In [39]:
exch_cycle = generate_exchange_data(CHAIN_LIQUIDITY_TOK_1, CHAIN_LIQUIDITY_TOK_2)

In [42]:
delta_1_prime = generate_upper_bounds(exch_cycle, x,y)
delta_1 = exch_cycle[0]['delta_in']

Exchange data: {'exch_number': 1, 'liq_in': 1.0, 'liq_out': 2.0, 'eta': 0.665331998665332, 'upper_bound': 0.665331998665332, 'delta_in': 0.5, 'delta_out': 0}
Exchange data: {'exch_number': 2, 'liq_in': 1.0, 'liq_out': 2.0, 'eta': 0.47238571905238574, 'upper_bound': 0.9447714381047715, 'delta_in': 0, 'delta_out': 0}
Exchange data: {'exch_number': 3, 'liq_in': 1.0, 'liq_out': 2.0, 'eta': 0.3353938605271939, 'upper_bound': 0.6707877210543878, 'delta_in': 0, 'delta_out': 0}
Exchange data: {'exch_number': 4, 'liq_in': 1.0, 'liq_out': 2.0, 'eta': 0.2381296409743077, 'upper_bound': 0.4762592819486154, 'delta_in': 0, 'delta_out': 0}


In [44]:
print(f"Profit calculation: \n")
print(f"\tDelta 1: {delta_1}")
print(f"\tDelta_1_prime: {delta_1_prime}")
print(f"\tProfit: {delta_1_prime - delta_1}")
print(f"\tReturn: {delta_1_prime/delta_1 - 1}")

Profit calculation: 

	Delta 1: 0.5
	Delta_1_prime: 0.4762592819486154
	Profit: -0.023740718051384613
	Return: -0.047481436102769226


In [ ]:
a